# Feature selection

In this example we will see how to **select features** through **univariate feature selection**. 

As a toy example, we will use data from 'Titanic: Machine Learning for Disaster', one of the most popular Kaggle competitions. However, we will not use the original data set. We will use a modified data set, which results from a [Kaggle kernel](https://www.kaggle.com/pmarcelino/data-analysis-and-feature-extraction-with-python) that I did. 

[Here](https://github.com/pmarcelino/blog/data/titanic_modified.csv) you can access to the data set used in this exercise.

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Load data

Let's load the data set.

In [2]:
df = pd.read_csv('data/titanic_modified.csv', index_col=0)  # You may have to modify this 
df.head()

,Survived,Fare,FamilySize,Imputed,Pclass_2,Pclass_3,Sex_male,Age_Child,Age_Elder,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Other
0,0,7.2500,1,0,0,1,1,0,0,0,1,0,1,0,0
1,1,71.2833,1,0,0,0,0,0,0,0,0,0,0,1,0
2,1,7.9250,0,0,0,1,0,0,0,0,1,1,0,0,0
3,1,53.1000,1,0,0,0,0,0,0,0,1,0,0,1,0
4,0,8.0500,0,0,0,1,1,0,0,0,1,0,1,0,0


As I mentioned, this data set results from a [kernel](https://www.kaggle.com/pmarcelino/data-analysis-and-feature-extraction-with-python) that I already did for the Titanic competition. I'll summarize each of the features to give you some context:

* **Survived**. Target variable. It's 1 if the passenger survived and 0 if it didn't.
* **Fare**. Passenger fare. It keeps the same properties as the original feature.
* **FamilySize**. It's the sum of the original features SipSp and Parch. SipSp refers to the # of siblings / spouses aboard the Titanic. Parch refers to the # of parents / children aboard the Titanic.
* **Imputed**. Identifies instances where some missing data imputation was made.
* **Pclass**. Ticket class. The feature was [encoded](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features), so we just have the features corresponding to the second and third class. The one corresponding to the first class was deleted to avoid the [dummy trap](http://www.algosome.com/articles/dummy-variable-trap-regression.html).
* **Sex**. It was also encoded. It's 1 if the instance corresponds to a male, and 0 if it corresponds to a female.
* **Age**. Encoded to the following classes: Children, Adult, and Elder. It's an Adult if Age_Child = 0 and Age_Elder = 0.
* **Embarked**. Port of embarkation. Originally, we had three possible ports: C = Cherbourg, Q = Queenstown, S = Southampton.
* **Title**. Results from the name of the passenger. Guess what? It's also an encoded feature.

Speaking of encoded features, they are a good example of how feature selection is useful. For example, when we do [one-hot encoding](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), a lot of features tend to be created (*n*-1 features to be more precise, where *n* is the number of existing class values). Accordingly, these cases are prone to the application of feature selection.

# Train and test data sets

In this example we will be applying different feature selection methods. The usefulness of these methods depend on their capacity to improve the performance of our prediction models. 

To test the models' performance on unseen data, we will need to have train and test data sets.

In [3]:
# Create train and test set
from sklearn.model_selection import train_test_split

X = df.drop('Survived', axis=1)  # Keep all features except 'Survived'
y = df['Survived']  # Just keep 'Survived'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

Brief note on *test_size=0.20*: 

* It's common practice to consider 80% of your data for training and 20% for testing. However, you can test different ratios and see how it impacts the model. From my experience, you don't get much from tuning this parameter. If you want to know more about this topic, there's an interesting thread on [StackOverflow](https://stackoverflow.com/questions/13610074/is-there-a-rule-of-thumb-for-how-to-divide-a-dataset-into-training-and-validatio).

# Feature selection

We will use scikit-learn to perform feature selection. In the [official documentation](http://scikit-learn.org/stable/modules/feature_selection.html), you have all the details about their feature selection methods. Here, I'll make it easy for you. 

To perform feature selection, you need:

1. To define the **number of features** that you want to keep.
1. To select the **scoring function** that will evaluate the relationship between the variables.

In what concerns the number of features, you can define it through:

1. **SelectKBest**. Selects the *k* best features.
1. **SelectPercentile**. Selects the best features into the percentile that you define.

Regarding the scoring functions, you'll have different functions for classification and regression problems. Since we are dealing with a classification problem, I'll just list the two scoring functions that I use the most:

1. **f_classification**. Based on analysis of variance (ANOVA).
1. **mutual_info_classification**. Based on mutual information.

I'll not go into the details on each scoring function, but I'll set up the system so that we can test both functions and keep the one that performs better. Brute force style.


![](figures/meme_brute_force.jpg)

## f_classification

To apply this feature selection method, we need to:

* Define if we will use SelectKBest or SelectPercentile. In this case, since we have few features, I'll go for **SelectKBest** and I'll test all possible *k* values to choose the best one.
* Select the machine learning algorithm that we want to apply. For the sake of simplicity, I'll use a **logistic regression**. It will run fast and it can provide us some insights about the data.
* Choose a performance metric. Since we are solving a Kaggle competition, we will use the performance metric defined by them. In this case, the performance metric is **accuracy** (percentage of correct predictions).

Now, we just need to build a function that runs for all *k* values, applies logistic regression, and computes the accuracy of the model. Then, we keep the *k* value that tell us the features we need to keep to optimize the performance of the model.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

lr = LogisticRegression()
max_score = 0

for i in range(1, np.shape(X_train)[1]+1):
    # Feature selection
    select = SelectKBest(f_classif, k=i)
    select.fit(X_train, y_train)  # Computes the statistical relationship between the features and the output
    X_train_selected = select.transform(X_train)  # Reduces the number of features to k
    
    # Model selection
    lr.fit(X_train_selected, y_train)
    strat_kfold = StratifiedKFold(10, random_state=7)
    score = cross_val_score(lr, X_train_selected, y_train, scoring='accuracy', cv=strat_kfold)
    print('CV accuracy: %.3f +/- %.3f' % (np.mean(score), np.std(score)))

    # Optimal number of features
    if np.mean(score) > max_score:
        max_score = np.mean(score)
        best_n_features = i
        mask = select.get_support(indices=True)

print('\nThe optimal number of features is: %i' % best_n_features)
print('The maximum score is: %.3f' % max_score)
print('The optimal features are: ', mask)

CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.800 +/- 0.059
CV accuracy: 0.803 +/- 0.060
CV accuracy: 0.803 +/- 0.058
CV accuracy: 0.812 +/- 0.051
CV accuracy: 0.810 +/- 0.050
CV accuracy: 0.813 +/- 0.054
CV accuracy: 0.809 +/- 0.052
CV accuracy: 0.834 +/- 0.051
CV accuracy: 0.834 +/- 0.051
CV accuracy: 0.831 +/- 0.054

The optimal number of features is: 12
The maximum score is: 0.834
The optimal features are:  [ 0  1  2  3  4  5  6  9 10 11 12 13]


Brief note about cross-validation:

* When we want to estimate the performance of a model in a more stable way, we usually do [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html). It is common to use a specific cross-validation version called [k-fold cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold). 
* When we are dealing with classification problems, I recommend you to use [stratified k-fold cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold). This will allow your *k* folds to keep the same proportion of class values. In regression problems you don't have this issue.

## mutual_info_classif

We will follow the same logic as *f_classif*. 

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

lr = LogisticRegression()
max_score = 0

for i in range(1, np.shape(X_train)[1]+1):
    # Feature selection
    select = SelectKBest(mutual_info_classif, k=i)
    select.fit(X_train, y_train)  # Computes the statistical relationship between the features and the output
    X_train_selected = select.transform(X_train)  # Reduces the number of features to k
    
    # Model selection
    lr.fit(X_train_selected, y_train)
    strat_kfold = StratifiedKFold(10, random_state=7)
    score = cross_val_score(lr, X_train_selected, y_train, scoring='accuracy', cv=10)
    print('CV accuracy: %.3f +/- %.3f' % (np.mean(score), np.std(score)))

    # Optimal number of features
    if np.mean(score) > max_score:
        max_score = np.mean(score)
        best_n_features = i
        mask = select.get_support(indices=True)

print('\nThe optimal number of features is: %i' % best_n_features)
print('The maximum score is: %.3f' % max_score)
print('The optimal features are: ', mask)

CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.802 +/- 0.047
CV accuracy: 0.799 +/- 0.047
CV accuracy: 0.800 +/- 0.048
CV accuracy: 0.800 +/- 0.048
CV accuracy: 0.800 +/- 0.048
CV accuracy: 0.807 +/- 0.052
CV accuracy: 0.836 +/- 0.052
CV accuracy: 0.809 +/- 0.047
CV accuracy: 0.827 +/- 0.052
CV accuracy: 0.836 +/- 0.046
CV accuracy: 0.831 +/- 0.053
CV accuracy: 0.830 +/- 0.052
CV accuracy: 0.831 +/- 0.054

The optimal number of features is: 8
The maximum score is: 0.836
The optimal features are:  [ 0  1  4  5 10 11 12 13]


# Conclusion

The methods give us different results. *mutual_info_classif* give us a higher performance score and a lower number of features. This suggests that *mutual_info_classif* would be a better choice for this problem. It has a higher accuracy score (better performance) and less features (less computational cost). 

We can also notice that some features are excluded by both methods. When different methods guide us to the same results, that means something. Try to use these insights to increase your knowledge about the data and the problem.

# Extra

Before we finish, let's go DRY. DRY is a principle of software development which stands for 'don't repeat yourself'. This means that we should aim at reducing repetition of code. 

You probably noticed that we used the code structure to apply both methods. Here, I'll give you a code snippet that you can reuse in generic problems.

In [6]:
def select_features_univariate(X, y, scoring_function=mutual_info_classif, scoring_performance='accuracy'):
    """Function to select features using univariate methods

    Assumptions:
        - You want to use SelectKBest
        - You're dealing with a classification problem
        - You want to use f_classif or mutual_info_classif as scoring functions
        - You believe that this can be useful :P

    Args:
        X (pd.DataFrame): Input values
        y (pd.Series): Output value
        scoring_function (str): Function that scores relationships between variables
        scoring_performance (str): Performance metric

    Returns:
        best_n_features: Optimal number of features
        max_score: Maximum performance score
        mask: Optimal set of features

    """
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import StratifiedKFold
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_classif
    from sklearn.feature_selection import mutual_info_classif
    
    max_score = 0
    lr = LogisticRegression()

    for i in range(1, np.shape(X)[1]+1):
        # Feature selection
        select = SelectKBest(scoring_function, k=i)
        select.fit(X, y)  # Computes the statistical relationship between the features and the output
        X_selected = select.transform(X)  # Reduces the number of features to k

        # Model selection
        lr.fit(X_selected, y)
        strat_kfold = StratifiedKFold(10, random_state=7)
        score = cross_val_score(lr, X_selected, y_train, scoring=scoring_performance, cv=strat_kfold)
        print('CV score: %.3f +/- %.3f' % (np.mean(score), np.std(score)))

        # Optimal number of features
        if np.mean(score) > max_score:
            max_score = np.mean(score)
            best_n_features = i
            mask = select.get_support(indices=True)

    print('\nThe optimal number of features is: %i' % best_n_features)
    print('The maximum score is: %.3f' % max_score)
    print('The optimal features are: ', mask)
    
    return (best_n_features, max_score, mask)